## Conversation Patterns


> **AutoGen > Tutorial > Conversation Patterns**
https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns


# Conversation Patterns

이전 챕터에서는 두 개의 에이전트가 대화하는 Two-agent 대화 패턴을 사용했고, 이를  `initiate_chat` 메서드를 통해 시작했다.
**Two-Agent** 대화는 매우 유용한 대화 패턴이지만, AutoGen은 이보다 더 강력한 기능을 제공한다. 이번 챕터에서는 먼저 **Two-Agent** 대화 패턴과 그 결과(chat result)를 조금 더 자세히 살펴보고, 이후에 **두 개 이상의 에이전트가 참여하는 다양한 대화 패턴**을 소개한다.

## An Overview

[1] **`Two-Agent chat`**
- 가장 기본적인 대화 패턴, 두 에이전트가 서로 메시지를 주고 받는 형태

[2] **`Sequential Chat`**
- 두 에이전트 간의 대화를 연속적으로 연결하는 패턴
이 때 `carryover 메커니즘`을 통해 이전 대화의 요약(summary)를 다음 대화의 컨텍스트로 전달할 수 있다. 

[3] **`Group Chat`**
- **두 명 이상의 에이전트**가 동시에 참여하는 단일 대화
- Group Chat에서 중요한 포인트는 **다음에 말할 에이전트를 어떻게 선택할 것인가?** 이다. 이를 위해서 다양한 방식이 제공된다.
- **다음 발화자 선택 전략**
	
    - `round_robin` : 순차적으로 에이전트 선택
    - `random` : 무작위 선택
    - `manual` : 사용자가 직접 선택
    - `auto` : 기본값, LLM이 자동으로 판단하여 선택
    
- **다음 발화자에 대한 선택 조건 제한** : 특정 상황에 따라 말할 수 있는 에이전트를 제한할 수 있음 (예제 참고)
- **사용자 정의 발화자 선택 함수 제공 기능** : 이 기능을 사용하면 `StateFlow` 모델을 구성할 수 있다. 즉, 에이전트 간의 **결정적인(deterministric) 워크플로우**를 만들 수 있다. 자세한 내용은 해당 가이드 및 StateFlow 관련 블로그 글 참고 (StateFlow는 추후 User Guide 에서 Customize Speaker Selection 파트에서 언급)

> **StateFlow**
https://microsoft.github.io/autogen/0.2/blog/2024/02/29/StateFlow/


## Two-Agent Chat and Chat Result

두 에이전트 채팅은 가장 간단한 형태의 대화 패턴이다.

![](https://velog.velcdn.com/images/heyggun/post/a3ca793a-8353-4c56-aa04-85772e7b6fcb/image.png)

위의 그림은 **Two-Agent** 구조도를 보여주는 다이어그램이다.
이 구조는 LLM(대형 언어 모델)을 활용해서 두 개의 에이전트가 협업하여 질문에 대한 답변을 생성하는 시스템이다. 각 구성 요소는 아래와 같이 해석 할 수 있다.

사용자가 질문을 입력하면(`Message` : "What is triangle inequality?"), 이 질문과 질문을 처리하기 위한 배경 정보나 세부 설정인 컨텍스트(`Context` : "summary_method: reflection_with_llm") 가 시스템에 전달된다. 시스템 내부에는  `Initializer(초기화기)`, `Agent A(에이전트 A)`, `Agent B(에이전트 B)`, `Summarizer(요약기)` 가 작동한다. 이 구조를 통해 두 에이전트가 대화하며 정보를 정제하고, 마지막에 요약된 답변을 생성한다. 

`Initilaizer` 는 사용자 질문과 컨텍스트를 바탕으로 초기 메시지를 생성하여 `Agent A`에게 전달한다. `Agent A`와 `Agent B`는 서로 번갈아 대화하면서 질문에 대해 논의하고 정보를 확장한다. 이 대화를 통해 풍부한 내용의 히스토리를 생성한다. `Summarizer`는 `Agent A`와 `Agent B`의 대화 내용을 기반으로 핵심 정보를 추출하고 요약해 사용자에게 전달할 **Chat Result**를 생성한다.

즉 데이터의 흐름이 사용자 입력(Message) -> Initalizer -> Agent A <-> Agent B -> 대화 히스토리 -> Summarizer -> Chat result 인 셈이다.
이 구조는 일반적인 단일 LLM 응답보다 더 협업적이고 심화된 응답 생성을 목표로 한다. 특히 사고 기반 요약(reflection with LLM) 같은 고급 전략에 적합하다.




--- 

Two-Agent 대화는 두 가지 입력을 받는다.
- `message`: 호출자가 제공하는 문자열 메시지
- `context` : 대화에 사용될 다양한 파라미터들을 정의하는 컨텍스트

보내는 에이전트(`sender agent`)는 `chat initializer` 메서드(**ConversableAgent의 generate_init_message 메서드**)를 사용해 입력값으로부터 초기 메시지를 생성하고, 이 메시지를 받는 에이전트(`recipient agent`)에게 전달함으로써 대화를 시작한다. 여기서 **sender agent**는 `initiate_chat` 메서드가 호출된 에이전트를 의미하며, **recipient agent**는 상대방 에이전트를 뜻한다.

대화가 종료되면 대화 기록(`chat history`)는 `Chat Summarizer`에 의해 처리된다. **Summarizer**는 전체 대화 내용을 요약하고, 토큰 사용량(token usage)을 계산한다. 요약 방식은 `initiate_chat` 메서드의 `summary_method` 파라미터를 통해 설정할 수 있으며, 기본값은 마지막 메시지를 요약으로 사용하는 방식이다. **`summary_method='last_msg')`**

아래 예시는 student agent(학생 에이전트)와  teacher agent(교사 에이전트) 간의 Two-agent 대화이다. 이 대화에서는 LLM 기반 요약 방식을 사용하는 **summarizer**가 설정되어 있다. 

In [2]:
import os
from config import settings
from autogen import ConversableAgent

api_key = settings.openai_api_key.get_secret_value()

In [3]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

In [6]:
student_agent = ConversableAgent(
    name="Student_Agent",
    system_message="You are a student willing to learn.",
    llm_config=llm_config
    
)

teacher_agent = ConversableAgent(
    name="Teacher_Agent",
    system_message="You are a math teacher.",
    llm_config = llm_config,
)

chat_result = student_agent.initiate_chat(
    teacher_agent,
    message="What is triangle inequlaity?",
    summary_method = "reflection_with_llm",
    max_turns=2
)

Student_Agent (to Teacher_Agent):

What is triangle inequlaity?



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Teacher_Agent (to Student_Agent):

The triangle inequality is a fundamental theorem in geometry that describes a property of triangles. It states that for any triangle, the sum of the lengths of any two sides must be greater than the length of the remaining side. 

This can be formulated mathematically as follows:

For any triangle with sides of lengths \( a \), \( b \), and \( c \):

1. \( a + b > c \)
2. \( a + c > b \)
3. \( b + c > a \)

If any of these inequalities do not hold, then the side lengths cannot form a triangle. The triangle inequality is important in various fields, including mathematics, physics, and computer science.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Student_Agent (to Teacher_Agent):

Thank you for the explanation! Can you provide an example of how to use the triangle inequality i

In [8]:
from pprint import pprint

In [9]:
pprint(chat_result.summary)

('The triangle inequality states that for any triangle with sides of lengths '
 '\\( a \\), \\( b \\), and \\( c \\), the following conditions must hold: \\( '
 'a + b > c \\), \\( a + c > b \\), and \\( b + c > a \\). If all conditions '
 'are satisfied, the lengths can form a triangle; if any condition fails, they '
 'cannot. Examples demonstrated this principle with both successful and '
 'unsuccessful cases of forming triangles.')


('삼각형 부등식은 변의 길이가 \\( a \\), \\( b \\), 그리고 \\( c \\)인 모든 삼각형에 대해 다음 조건이 충족되어야 함을 나타냅니다. \\(a + b > c \\), \\( a + c > b \\), 그리고 \\( b + c > a \\). 모든 조건이 충족되면 변의 길이는 삼각형을 형성할 수 있으며, 어떤 조건이든 충족되지 않으면 삼각형을 형성할 수 없습니다. 다음 예시는 삼각형 형성의 성공 및 실패 사례에서 이 원리를 보여줍니다.')

라는 답변이 나오게 된다. 

위 예제에서는 `summary_method`가 **`reflection_Wht_llm`** 으로 설정되어 있으며, 이 방법은 대화 메시지 리스트를 받아 LLM 호출을 통해 요약을 생성한다.
요약 방식은 다음과 같은 순서로 LLM을 사용한다.

(1) 먼저 **받는 에이전트(`recipient`)**의 LLM을 사용 시도 한다.
(2) 사용할 수 없는 경우 **보내는 에이전트(`sender`)의 LLM을 사용한다.

이 예제에서는 `Teacher_Agent`가 **recipient**, `Studnet_Agent`가 **sender** 이다.

In [10]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


LLM에 전달되는 입력 프롬프트는 아래와 같은 `default prompt(기본 프롬프트)`이다. 원하는 경우 `initiate_chat`의 `summary_prompt` 인자를 사용해 **커스텀 프롬프트를 설정할 수도 있다.**

In [12]:
pprint(chat_result.chat_history)

[{'content': 'What is triangle inequlaity?',
  'name': 'Student_Agent',
  'role': 'assistant'},
 {'content': 'The triangle inequality is a fundamental theorem in geometry '
             'that describes a property of triangles. It states that for any '
             'triangle, the sum of the lengths of any two sides must be '
             'greater than the length of the remaining side. \n'
             '\n'
             'This can be formulated mathematically as follows:\n'
             '\n'
             'For any triangle with sides of lengths \\( a \\), \\( b \\), and '
             '\\( c \\):\n'
             '\n'
             '1. \\( a + b > c \\)\n'
             '2. \\( a + c > b \\)\n'
             '3. \\( b + c > a \\)\n'
             '\n'
             'If any of these inequalities do not hold, then the side lengths '
             'cannot form a triangle. The triangle inequality is important in '
             'various fields, including mathematics, physics, and computer '
          

In [13]:
pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 628,
                                                                 'cost': 0.0005273999999999999,
                                                                 'prompt_tokens': 1004,
                                                                 'total_tokens': 1632},
                                      'total_cost': 0.0005273999999999999},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 628,
                                                                 'cost': 0.0005273999999999999,
                                                                 'prompt_tokens': 1004,
                                                                 'total_tokens': 1632},
                                      'total_cost': 0.0005273999999999999}}


## Sequential Chats

이 패턴의 이름은 **두 에이전트 간의 대화를 일련의 단계로 연결한 것**이다.
`carryover`라는 메커니즘을 통해 이전 대화의 요약(summary)이 다음 대화의 컨텍스트로 전달된다.
이 패턴은 **복잡한 작업을 사옿 의존적인 하위 작업들로 나누어 처리** 할 때 유용하다. 
아래 그림은 이 패턴이 어떻게 동작하는지를 시작적으로 보여준다. 

![](https://velog.velcdn.com/images/heyggun/post/273e31d8-3058-4af9-bbc2-2667087c644b/image.png)

다단계 에이전트 협업 구조를 나타내는 그림으로 여러 개의 두 에이전트간 `Chat Block(대화 블록)`이 순차적으로 이어지며, 각 블록은 외부에서 주어진 `Context(문맥)`과 `Message(메시지)`, 그리고 이전 블록의 결과물인 `Carryover`를 입력으로 받아 처리한다.
각 블록에는 두 개의 에이전트가 포함되어 있고, 이들은 내부적으로 서로 Chat(대화) 하면서 주어진 입력을 처리한다.

구성 요소를 간략하게 설명해보자면 
- **Context** : 외부에서 주어진 상황 정보, 각 블록마다 독립적으로 주어짐
- **Message** : 각 단계에 주어진 입력 메시지
- **Carryover** : 이전 블록에서 생성된 결과. 다음 블록으로 전달되어 연속적인 문맥 유지를 가능하게 함
- **Agent A** : 모든 블록에 공통적으로 포함된 기본 에이전트. 핵심 역할을 담당할 가능성이 높음
- **Agent B~E** : 블록별로 상이한 역할을 수행하는 파트너 에이전트. 각각의 역할이 다를 수 있음
- **Chat** : 각 블록 내부에서 두 에이전트 간의 상호작용. 메시지와 문맥을 기반으로 논의하고 결과 도출. 

으로 구성되어 있다.

흐름으로 `context` + `message`가 Agent A와 B에게 주어지면, 둘이 Chat을 통해 결과를 생성한다. 이 결과는 Carryover로 다음 블록으로 전달되고, 이후 블록들은 각 블록마다 새로운 Context와 Message가 주어지고 이전 블록의 Carryover가 함께 입력된다. Agent A는 동일하지만 Agent C, D, E로 변경되고 있고 내부에서 Chat -> 결과 생성 -> Carryover 전송을 반복한다.


In [14]:
from autogen import ConversableAgent
from config import settings

api_key = settings.openai_api_key.get_secret_value()

In [15]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key,
            }
        ]
}

In [29]:
number_agent = ConversableAgent(
    name= "Number_Agent",
    system_message= "You return me the numbers I give you, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

adder_agent = ConversableAgent(
    name="Adder_agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one numb er each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

multiplier_agent= ConversableAgent(
    name="Multiplier_Agent",
    system_message="You mulitply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new number, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config =llm_config,
    human_input_mode="NEVER",
)


In [30]:
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient" : adder_agent,
            "message" : "14",
            "max_turns" : 2,
            "summary_method" : "last_msg",
        },
        {
            "recipient" : multiplier_agent,
            "message" : "These are my numbers",
            "max_turns" : 2,
            "summary_method" : "last_msg",
            
        },
        {
            "recipient" : subtracter_agent,
            "message" : "These are my numbers",
            "max_turns" : 2,
            "summary_method" : "last_msg",
            
        },
        {
            "recipient" : divider_agent,
            "message" : "Thse are my numbers",
            "max_turns" : 2,
            "summary_method" : "last_msg",
            
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_agent):

14

--------------------------------------------------------------------------------
Adder_agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_agent):

15

--------------------------------------------------------------------------------
Adder_agent (to Number_Agent):

16

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (57671dc8-c791-4045-8773-bb51e03efa38): Maximum turns (2) reached

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Contex

In [31]:
print("First Chat", chat_result[0].chat_history[0]['content'])
print("Second Chat", chat_result[1].chat_history[0]['content'])
print("Third Chat", chat_result[2].chat_history[0]['content'])
print("Fourth Chat", chat_result[3].chat_history[0]['content'])

First Chat 14
Second Chat These are my numbers
Context: 
16
Third Chat These are my numbers
Context: 
16
32  
64
Fourth Chat Thse are my numbers
Context: 
16
32  
64
14  
30  
62  


In [28]:
print("First Chat Summary", chat_result[0].summary)
print("Second Chat Summary", chat_result[1].summary)
print("Third Chat Summary", chat_result[2].summary)
print("Fourth Chat Summary", chat_result[3].summary)

First Chat Summary 16
Second Chat Summary 32  
64
Third Chat Summary 14  
30  
62  
Fourth Chat Summary 4  
8  
16  
3.5  
7.5  
15.5  


In [35]:
chat_result

[ChatResult(chat_id=None, chat_history=[{'content': '14', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': '15', 'role': 'user', 'name': 'Adder_agent'}, {'content': '15', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': '16', 'role': 'user', 'name': 'Adder_agent'}], summary='16', cost={'usage_including_cached_inference': {'total_cost': 2.2949999999999995e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.2949999999999995e-05, 'prompt_tokens': 129, 'completion_tokens': 6, 'total_tokens': 135}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[]),
 ChatResult(chat_id=None, chat_history=[{'content': 'These are my numbers\nContext: \n16', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': '32', 'role': 'user', 'name': 'Multiplier_Agent'}, {'content': '16  \n32', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': '32  \n64', 'role': 'user', 'name': 'Multiplier_Agent'}], summary='32  \n64', cost={'usage_including_cached_inference': {'total_cost': 3.

## Group Chat

지금까지는 **두 개의 에이전트가 참여하는 대화 패턴**이나 **두 에이전트 간의 대화 스퀀스**만 살펴보았다. 하지만 AutoGen은 이보다 더 일반적인 대화 패턴인  **Group Chat**을 제공한다.  
**Group Chat**은 **두 명 이상의 에이전트가 하나의 대화 스레드 함께 참여하는 방식이다.**
모든 에이전트는 **하나의 공유된 컨텍스트(context)** 안에서 상호작용하며, 여러 에이전트 간의 협업이 필요한 복잡한 작업에 적합하다.

아래 그림은 **Group Chat**이 어떻게 동작하는지를 보여준다.

![](https://velog.velcdn.com/images/heyggun/post/dc94658b-a837-4711-b4e2-f65d7f86ee4b/image.png)


간략하게 그림을 설명해보면 **Multi-Agent System(멀티 에이전트 시스템)** 에서 **Group Chat Manager(그룹 채팅 관리자)**를 중심으로 에이전트 간 대화가 이루어진 과정을 3단계로 설명한 흐름도이다.

`Agent A`, `Agent B`, `Agent C`, `Agent D`는 발언 가능한 개별 AI 에이전트이고 `Group Chat Manager`는 발언 권한 관리 및 메시지 전달을 조율하는 중앙 관리자 역할이다.

단계별로 흐름을 보면 (1) `Select Speaker` : 발언자를 선정한다. *Group Chat Manager*는 네 명의 에이전트(A~D) 중에서 **한 명의 발언자(Speaker)**를 선택한다. 위 예시에서는 *Agent B*가 점선 박스로 강조되어 발언자로 선택된 것이다.

(2) `Agent Speaker` :  선택된 Agent B는 자신의 메시지를 *Group Chat Manager*에게 보낸다. 이 단계에서는 발언 자체가 *Group Chat Manager*에게 전달되고, 다른 에이전트는 아직 해당 메시지를 받지 못한 상태이다.

(3) `Brodcast Message` : 메시지 전파. *Group Chat Manager*는 받은 메시지를 모든 에이전트(A,B,C,D)dprp **브로드캐스트 방식**으로 전달한다. 이로써 모든 에이전트가 동일한 정보를 공유하게 되어, **공동의 문맥(Context)**가 유지된다.


---
**Group Chat**은 `GroupChatManager` 라는 특별한 타입의 에이전트에 의해 **orchestrate(조율)** 된다. Group Chat의 첫 단계는, **Group Chat Manager**가 발화할 에이전트를 선택한다. 선택된 에이트가 메시지를 생성하면, 그 메시지는 다시 Group Chat Manager에게 전달되고, Group Chat Manager는 해당 메시지를 **그룹 내의 다른 에이전트들에게 `broadcasts(브로드 캐스트, 공유)` 한다.** 이 과정은 대화가 종료될 때까지 반복된다.

Group Chat Manager는 다음 발화자를 선택하기 위해 여러 가지 전략을 사용할 수 있다. 현재 지원되는 전략은 다음과 같다.

- `round_robin`: 에이전트들이 설정된 순서에 따라 **순차적으로 돌아가며 선택**
- `random` : **무작위로 에이전트 선택**
- `manual` : **사용자(사람)**이 직접 에이전트 선택
- `auto` : Group Chat Manager의 LLM이 **자동으로 적절한 에이전트 선택**

이 패턴을 설명하기 위한 예시로, 이전 예제에서 사용했던 **산술 연산 에이전트들**을 다시 활용한다. 목표눈 **하나의 숫자를 여러 에이전트의 산술 연산을 통해 원하는 타겟 숫자로 변환하는 것**이다.

이 예제에서는 발화자 선택 전략으로 `auto` 전략을 선택한다
또한, Group Chat Manager가 적절한 에이전트를 선택할 수 있도록, 각 에이전트에 **명확한 description(설명)** 도 설정해준다. 
설명이 없을 경우, 시스템은 에이전트의 `system_message`를 기반으로 판단하게 되며, 이것은 최선의 선택이 아닐 수 있다.

In [36]:
from autogen import ConversableAgent
from config import settings

api_key =settings.openai_api_key.get_secret_value()

In [38]:
llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

In [39]:
number_agent = ConversableAgent(
    name= "Number_Agent",
    system_message= "You return me the numbers I give you, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

adder_agent = ConversableAgent(
    name="Adder_agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one numb er each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

multiplier_agent= ConversableAgent(
    name="Multiplier_Agent",
    system_message="You mulitply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new number, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config =llm_config,
    human_input_mode="NEVER",
)


In [41]:
adder_agent

In [42]:
adder_agent.description = "Add 1 to each input numbers."
multiplier_agent.description = "Multiply each input number by 2"
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."


In [45]:
adder_agent.description

'Add 1 to each input numbers.'

In [55]:
from autogen import GroupChat

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent,
            divider_agent, number_agent],
    messages=[],
    max_round=10,
)

In [56]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

In [57]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

4  
5  
6  
7  
8  
9  
10  
11  
12  
13  

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

4  
5  
6  
7  
8  
9  
10  
11  
12  
13  

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

2  
3  
4  
5  
6  
7  
8  
9  
10  
11  

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6  
8  
10  
12  
14  
16  
18  
20  
22  
24  

--------------------------------------------------------------------------------

Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

1.5  
2  
3 

In [51]:
pprint(chat_result.summary)

('The conversation involves different agents providing sequential numbers that '
 'can be generated from the starting number 3, with the goal of reaching the '
 'number 13 through addition, and various other operations such as '
 'subtraction, multiplication, and division producing respective ranges of '
 'outcomes.')


In [53]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'My number is 3, I want to turn it into 13.', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': '4  \n5  \n6  \n7  \n8  \n9  \n10  \n11  \n12  \n13  ', 'name': 'Adder_agent', 'role': 'user'}, {'content': '4  \n5  \n6  \n7  \n8  \n9  \n10  \n11  \n12  \n13  ', 'name': 'Adder_agent', 'role': 'user'}, {'content': '2  \n3  \n4  \n5  \n6  \n7  \n8  \n9  \n10  \n11  ', 'name': 'Subtracter_Agent', 'role': 'user'}, {'content': '6  \n8  \n10  \n12  \n14  \n16  \n18  \n20  \n22  \n24  ', 'name': 'Multiplier_Agent', 'role': 'user'}, {'content': '1.5  \n2  \n3  \n3.5  \n4  \n4.5  \n5  \n5.5  \n6  \n6.5  ', 'name': 'Divider_Agent', 'role': 'user'}], summary='The conversation involves different agents providing sequential numbers that can be generated from the starting number 3, with the goal of reaching the number 13 through addition, and various other operations such as subtraction, multiplication, and division producing respective ranges 

    지능 문제로 gpt-4o로 변경해봄

In [58]:
llm_config_2= {
    "config_list":
        [
            {
                "model" : "gpt-4o",
                "api_key" : api_key
            }
        ]
}

In [59]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config_2,
)

In [60]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

4  
5  
6  
7  
8  
9  
10  
11  
12  
13  

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

4  
5  
6  
7  
8  
9  
10  
11  
12  
13  

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

8  
10  
12  
14  
16  
18  
20  
22  
24  
26  

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

2  
3  
4  
5  
6  
7  
8  
9  
10  
11  

--------------------------------------------------------------------------------

Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

1.5  
2.0

### Send Introductions

에이전트에게 소개 메시지 보내기.

이전 예제에서는 *Group Chat Manager*가 다음 발화자를 선택할 수 있도록 각 에이전트에 `description` 을 설정했다. 하지만 이 설정은 *Group Chat Manager*에게만 도움이 될 뿐, 대화에 참여하는 다른 에이전트들이 서로를 알 수 있도록 하지는 않는다. 경우에 따라 **각 에이전트가 그룹 내 다른 에이전트에게 스스로 소개하는 것이 유용할 수 있다.**
이 기능은 `send_introductions=True`로 설정하면 사용할 수 있다.

In [65]:
from autogen import GroupChat

group_chat_with_introductions= GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent,
            divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

In [66]:
from autogen import GroupChatManager

group_chat_manager = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config=llm_config,
)

In [67]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

4  
14

--------------------------------------------------------------------------------

Next speaker: Multiplier_Agent

Multiplier_Agent (to chat_manager):

6  
13

--------------------------------------------------------------------------------

Next speaker: Subtracter_Agent

Subtracter_Agent (to chat_manager):

2

--------------------------------------------------------------------------------

Next speaker: Divider_Agent

Divider_Agent (to chat_manager):

1.5

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

4  
14  
6  
13  
2  
1.5  

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (c0014d07-c0d2-408d-866b-15e28e2

In [68]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'My number is 3, I want to turn it into 13.', 'role': 'assistant', 'name': 'Number_Agent'}, {'content': 'Hello everyone. We have assembled a great team today to answer questions and solve tasks. In attendance are:\n\nAdder_agent: Add 1 to each input numbers.\nMultiplier_Agent: Multiply each input number by 2\nSubtracter_Agent: Subtract 1 from each input number.\nDivider_Agent: Divide each input number by 2.\nNumber_Agent: Return the numbers given.', 'role': 'user', 'name': 'chat_manager'}, {'content': '4  \n14', 'name': 'Adder_agent', 'role': 'user'}, {'content': '6  \n13', 'name': 'Multiplier_Agent', 'role': 'user'}, {'content': '2', 'name': 'Subtracter_Agent', 'role': 'user'}, {'content': '1.5', 'name': 'Divider_Agent', 'role': 'user'}, {'content': '4  \n14  \n6  \n13  \n2  \n1.5  ', 'role': 'assistant', 'name': 'Number_Agent'}], summary='The conversation involves various operations to transform the number 3 into 13, using addition (

내부적으로는 **Group Chat**이 시작되기 전에 *Group Chat Manager*가 **각 에이전트의 이름과 설명이 포함된 메시지를 그룹 내 모든 에이전트에게 전달하여 서로를 소개하는 방식으로 작동한다. 

### Group Chat in a Sequential Chat

순차 대화(Sequential Chat)에서의 그룹 채팅 사용
-**Group Chat**은 `Sequential Chat)` 일부로도 사용할 수 있다.
이 경우, *Group Chat Manager*는 일반적인 에이전트처럼 **시퀀스 내의 하나의 에이전트로 간주**되어 동작한다.

In [72]:
from autogen import GroupChat
from autogen import GroupChatManager
from config import settings

api_key = settings.openai_api_key.get_secret_value()

llm_config = {
    
        "config_list":
            [
                {
                "model" : "gpt-4o-mini",
                "api_key" : api_key,
                
            }
            ]
    }

group_chat_wiht_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config=llm_config,
)

In [75]:
chat_result = group_chat_manager_with_intros.initiate_chats(
    [
        {
            "recipient" : group_chat_manager_with_intros,
            "message" : "My number is 3, I want to turn it into 13."
        },
        {
            "recipient" : group_chat_manager_with_intros,
            "message" : "Trun this number to 32."
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
chat_manager (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------


/Users/geonheekim/Library/Caches/pypoetry/virtualenvs/user-prediction-jY4gRI11-py3.11/lib/python3.11/site-packages/autogen/agentchat/chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



Next speaker: Adder_agent

Adder_agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

8

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (110200a3-6aa2-41a3-8c4e-c1d0b6a3785a): Maximum rounds (6) reached

********************************************************************************
Starting a new chat....

********************************************************************************

### Contrained Speaker Selection

발화자 선택 제약

*Group Chat* 은 강력한 대화 패턴이지만, **참여하는 에이전트 수가 많아질수록 발화자 제어가 어려워질 수 있다.** 이를 해결하기 위해 AutoGen은 **다음 발화자 선택을 제약할 수 있는 기능**을 제공한다.
바로`GroupChat` 클래스의 `allowed_or_disallowed_spekaer_transitions` 인자를 사용하는 것이다.

`allowed_or_disallowed_spekaer_transitions`는 **특정 에이전트 이후에 발화할 수 있는(또는 발화할 수 없는) 에이전트들의 목록을 정의한 딕셔너리**이다.
그리고 `speaker_transitions_type`인 자를 통해, 해당 제약이 **`허용 목록(allow)`**인지 **`비허용 목록(disallowed)`** 인지 명시할 수 있다.

발화자 선택 제약 예시는 아래와 같다.

In [76]:
allowed_transitions={
    number_agent: [adder_agent, number_agent],
    adder_agent : [multiplier_agent, number_agent],
    subtracter_agent : [divider_agent, number_agent],
    multiplier_agent : [subtracter_agent, number_agent],
    divider_agent : [adder_agent, number_agent],
}

위의 예시는 *Number Agent* 다음에는 *Adder Agwent*와 *Number Agent*만 발화 가능
*Adder Agent* 다음에는 *Multiplier Agent*와 *Number Agent*aks 발화 가능 등등 으로 되어 있는 예시이다.

이 설정들을 **Group Chat**에 적용하고, `speaker_transitions_type="allowed"`로 지정하면 이 조건들이 **양의 제약(positive constraints)** 으로 작동하게 된다.


In [77]:
from autogen import GroupChat
from autogen import GroupChatManager
from config import settings

api_key = settings.openai_api_key.get_secret_value()


In [78]:
llm_config= {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

In [82]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages = [],
    max_round=12,
    send_introductions=True,
    
)

constrained_group_chat_manager= GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config=llm_config,
)

chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm"
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------



Next speaker: Adder_agent

Adder_agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

7

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

8

--------------------------------------------------------------------------------

Next speaker: Number_Agent

Number_Agent (to chat_manager):

9

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

10

------------------------------------------------------------------

## Nested Chats (중첩 대화)

지금까지 본 대화 패턴들인 **Two-Agent Chat(두 에이전트 간 대화)**, **Sequential Chat(순차 대화)**, **Group Chat(그룹 대화)** 은 복잡한 워크플로우를 구성하는데 매우 유용하다.
하지만 이런 패턴들은 **단일 대화 인터페이스를 제공하지 않기 때문에**, **질문-응답형 봇**이나 **개인 비서** 같은 시나리오에는 적합하지 않을 수 있다.
또한 어떤 경우에는, 복잡한 워크플로우를 **하나의 에이전트로 캡슐화하여 다른 워크플로우에서 재사용**하는 것이 유리할 수 있다. 
AutoGen은 이러한 요구를 **Nested Chats(중첩 대화) 기능** 으로 지원한다.

**Nested Chats**은 `ConversableAgent`의 플러그인형 구성 요소인 **`nested_chats_handler`**를 통해 동작한다. 아래 그림은 메시지가 도착했을 때, *nested chats handler*가 **일련의 중첩 대화 시퀀스**를 어떻게 트리거하는지를 보여준다.

![](https://velog.velcdn.com/images/heyggun/post/a16a5ccc-4ff4-448d-849b-ce2b603a178e/image.png)

위 그림은 **Nested Chat(중첩 대화)** 구조에서의 **에이전트 간 상호작용 흐름**을 시각화한 다이어그램이다. 특히 한 에이전트(Agent A)가 필요에 따라 내부적으로 다른 에이전트가 대화를 생성하고, 그 결과를 통합해 응답하는 구조를 설명하고 있다.

전체적인 흐름을 보면 *Agent A*가 외부 메시지를 받고, 특정 조건(Trigger)에 따라 **중첩 대화(Nested Chats)**를 실행한다. 이 중첩 대화는 *Agent A*와 다른 에이전트들(예 : Agent B) 간의 **순차적 대화**로 구성되며, 결과는 다시 *Agent A*로 돌아와 최종 응답에 반영된다. 

구조적 구성요소를 설명해보면 *Agent A*의 내부에서는 사람이 개입할 수도 있는 상태(승인 또는 직접 조치하는) `human-in-the-loop`와 True일 때 중첩 대화를 수행하는 조건을 판별하는 `Trigger(조건 판별)`, 중첩 대화의 실행 및 결과 통합을 담당하는 `Nested Chats Handler`, 필요시 자동으로 처리하는 컴포넌트인 예를 들면 Code Executor, LLMs등과 같은 `Auto-reply components`가 있다. 

*Nested Chats(중첩 대화)*의 우측 영역에서는 *Agent A*와 *Agent B* 간의 다수의 **순차적 대화(Sequential Chats)**로 이루어진다. 각 대화는 `Agent A <-> Agent B` 쌍으로 이루어지고, 동일한 A와 다른 B들의 사이의 대화가 병렬적으로 진행된다. 

데이터의 흐름은
(1) Meesage->Agent A (외부로부터 메시지 수신)
(2) Trriger -> True (조건 충족 시 중첩 대화 실행)
(3) Message -> Nested Chats(Agent A가 중첩 대화 실행)
(4) Agent A <-> Agent B (내부 에이전트 간의 대화, 순차적 진행 가능)
(5) Chat Results -> Agent A (중첩 대화 결과 수신)
(6) Reply -> 외부 (Agent A가 최종 응답을 반환)

으로 볼 수 있다. 

---

메시지가 수신되고 `human-in-the-loop` 단계를 통과하면, nested chats handler는 사용자가 정의한 조건에 따라 **해당 메시지가 중첩 대화를 유발해야 하는지를 판단**한다. 조건이 충족되면, *handler*는 **Sequential Chat** 패턴을 이용한 **중첩 대화 시퀀스**를 시작한다.

각 중첩 대화에서는 **항상 동일한 에이전트(중첩 대화를 트리거한 에이전트)**가 발신자 역할을 하며, 중첩 대화가 끝나면 그 결과를 바탕으로 원래 메시지에 대한 응답을 생성한다. 기본적으로 **마지막 중첩 대화의 요약(summary)**가 응답으로 사용된다.

예시로 산술연산, 코드 기반 검증, 시(poem) 생성을 하나의 에이전트에 패키징한 *Arithmetic Agent*를 만들어본다. 
이 에이전트는 "숫자 3을 13으로 바꿔줘" 같은 요청을 받고, 그 변환 과정을 시로 표현한 결과를 반환한다.  우선, 산술 연산을 조율할 에이전트로 앞서 예제에서 사용한 `group_chat_manager_with_intros`를 재사용한다.

In [104]:
import tempfile

from config import settings
from autogen import ConversableAgent, GroupChat, GroupChatManager

api_key = settings.openai_api_key.get_secret_value()

llm_config = {
    "config_list":
        [
            {
                "model" : "gpt-4o-mini",
                "api_key" : api_key
            }
        ]
}

temp_dir = tempfile.gettempdir()

code_execution_config = {
    "use_docker" : False,
    "work_dir" : temp_dir
}



In [105]:
arithmetic_agent = ConversableAgent(
    name="Arithmetic_Agent",
    system_message= "",
    human_input_mode="ALWAYS",
    code_execution_config = code_execution_config,
)

code_writer_agent = ConversableAgent(
    name="Code_Writer_Agent",
    system_message="You are a code writer. You write Pytho nScript in Markdown code blocks.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

poetry_agent = ConversableAgent(
    name="Poetry_Agent",
    system_message = "You are an AI poet.",
    llm_config = llm_config,
    human_input_mode="NEVER"
)

In [106]:
number_agent = ConversableAgent(
    name= "Number_Agent",
    system_message= "You return me the numbers I give you, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

adder_agent = ConversableAgent(
    name="Adder_agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one numb er each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

multiplier_agent= ConversableAgent(
    name="Multiplier_Agent",
    system_message="You mulitply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new number, one number each line.",
    llm_config = llm_config,
    human_input_mode="NEVER",
)

divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config =llm_config,
    human_input_mode="NEVER",
)


In [107]:
adder_agent.description = "Add 1 to each input numbers."
multiplier_agent.description = "Multiply each input number by 2"
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

In [108]:
group_chat_wiht_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config=llm_config,
)

In [109]:
nested_chats = [
    {
        "recipient" : group_chat_manager_with_intros,
        "summary_method" : "reflection_with_llm",
        "summary_prompt" : "Summarize the sequence of operations used to turn the source number into target number.",
        
    },
    {
        "recipient" : code_writer_agent,
        "message" : "Write a Python script to verify the arithmetic operations is correct.",
        "summary_method"  : "reflection_with_llm",
    },
    {
        "recipient" : poetry_agent,
        "message" : "Write a poem about it.",
        "max_turns" : 1,
        "summary_method" : "last_msg",
        
    }
]

In [110]:
arithmetic_agent.register_nested_chats(
    nested_chats,
    trigger=lambda sender: sender not in [group_chat_manager_with_intros, code_writer_agent, poetry_agent],
)

In [111]:
reply = arithmetic_agent.generate_reply(
    messages=[
        {
            "role" : "user",
            "content" : "I have a number 3 and I want to turn it into 7."
        }
    ]
)


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

********************************************************************************
Starting a new chat....

********************************************************************************
Arithmetic_Agent (to chat_manager):

I have a number 3 and I want to turn it into 7.

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

4

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

5

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

6

--------------------------------------------------------------------------------

Next speaker: Adder_agent

Adder_agent (to chat_manager):

7

--------------------------------------------------------------------------------

Nex